In [ ]:
    threshold = np.linspace(1, 7, 100)
    additional_variable = np.linspace(0, 4, 100)

With different starting points, learning rates, gradient descent easily does not converge. Why???

With different arrays, animation got errors. Why???